In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pickle
import random
random.seed(42)
np.random.seed(42)

In [3]:
with open("../ressources/final_experiments/automode_datasets/df_leaveoneout_noagg.pickle", "rb") as file:
    original_df = pickle.load(file)

In [4]:
original_df.head()

,index,description,configuration,parameters,argos,behavior_tree,scores,avg_score,type,lightstype,arenatype,robotstype
0,0,The environment is a circular arena with radiu...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 --...,"[4.0, 2.0, 4.0, 5.0, 3.0, 4.0, 1.0, 3.0, 5.0, ...",3.600000,Foraging,TargetLights,CircularArena,CenteredSwarm
1,1,The environment consists of a circular arena w...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 1 --n0 0 --nchild0 2 --...,"[3.0, 3.0, 4.0, 2.0, 1.0, 2.0, 5.0, 3.0, 4.0, ...",2.900000,Foraging,UniformLights,CircularArena,CenteredSwarm
2,2,"With a radius of 3.44 meters, the circular are...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 --...,"[-8.57746, -4.79552, -7.16427, -9.41532, -6.30...",-7.018138,Connection,UniformLights,CircularArena,CenteredSwarm
3,4,The area is a rectangle with dimensions 5.11 x...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 --...,"[3.0, 5.0, 6.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, ...",4.700000,Foraging,TargetLights,RectangularArena,CenteredSwarm
4,7,The environment is a circle made out of 19 wal...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 --...,"[19.0, 19.0, 0.0, 14.0, 9.0, 15.0, 0.0, 13.0, ...",11.000000,Foraging,TargetLights,CircularArena,CenteredSwarm


In [5]:
(original_df.lightstype == "TargetLights").sum() / len(original_df)

np.float64(0.49023746701846965)

In [6]:
with open("../ressources/dpo_rl_model_25-01-20_15x100/dataset.pickle", "rb") as file:
    original_df = pickle.load(file)

In [7]:
original_df.head(3)

,llmin,configuration,parameters,argos,type,scores_bt1,scores_bt2,llmout_A,llmout_B,scores_A,scores_B,dataset_position
0,The environment features a circle composed of ...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Connection,-2.559813,-4.032016,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 0 -...,1.000000,0.972699,0
1,The rectangular area has dimensions 4.49 m x 7...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-7.834352,-8.19988,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.797862,0.787805,0
2,"With a radius of 3.54 meters, the circular are...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-16.24011,-13.42486,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,0.566587,0.644045,0


In [8]:
# Define the function to convert to SFT format
def convert_to_sft_format(row):
    if row["scores_bt1"] > row["scores_bt2"]:
        row["avg_score"] = row["scores_bt2"]
        row["behavior_tree"] = row["llmout_B"]
        row["llm_avg_score"] = row["scores_bt1"]
        row["llm_behavior_tree"] = row["llmout_A"]
    else:
        row["avg_score"] = row["scores_bt1"]
        row["behavior_tree"] = row["llmout_A"]
        row["llm_avg_score"] = row["scores_bt2"]
        row["llm_behavior_tree"] = row["llmout_B"]
    
    row["description"] = row["llmin"]
    
    return row

# Apply the function to the DataFrame
df = original_df.apply(convert_to_sft_format, axis=1)
df

,llmin,configuration,parameters,argos,type,scores_bt1,scores_bt2,llmout_A,llmout_B,scores_A,scores_B,dataset_position,avg_score,behavior_tree,llm_avg_score,llm_behavior_tree,description
0,The environment features a circle composed of ...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Connection,-2.559813,-4.032016,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 0 -...,1.000000,0.972699,0,-4.032016,--nroot 3 --nchildroot 3 --n0 0 --nchild0 0 -...,-2.559813,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,The environment features a circle composed of ...
1,The rectangular area has dimensions 4.49 m x 7...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-7.834352,-8.199880,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.797862,0.787805,0,-8.199880,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-7.834352,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,The rectangular area has dimensions 4.49 m x 7...
2,"With a radius of 3.54 meters, the circular are...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Distribution,-16.240110,-13.424860,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,0.566587,0.644045,0,-16.240110,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-13.424860,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,"With a radius of 3.54 meters, the circular are..."
3,"In this setting, a circular arena with a radiu...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Connection,-38.719570,-30.489020,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 0 -...,0.329445,0.482075,0,-38.719570,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,-30.489020,--nroot 3 --nchildroot 3 --n0 0 --nchild0 0 -...,"In this setting, a circular arena with a radiu..."
4,The rectangular area has dimensions 6.00 m x 3...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Aggregation,0.050000,0.000000,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 3 --n0 0 --nchild0 0 -...,0.070313,0.000000,0,0.000000,--nroot 3 --nchildroot 3 --n0 0 --nchild0 0 -...,0.050000,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,The rectangular area has dimensions 6.00 m x 3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,The environment features a rectangle with dime...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Connection,-5.006993,-6.662475,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.947679,0.915262,14,-6.662475,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-5.006993,--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 -...,The environment features a rectangle with dime...
1108,"A rectangular area, with a length of 4.26 mete...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",Foraging,0.100000,0.200000,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.043478,0.086957,14,0.100000,--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 -...,0.200000,--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,"A rectangular area, with a length of 4.26 mete..."
1109,The environment consists of a re

In [9]:
with open("../ressources/final_experiments/automode_datasets/df_increasing_size_validate.pickle", "rb") as file:
    original_df = pickle.load(file)
    
original_df[original_df.type=="Distribution"].avg_score.min()

np.float64(-29.58218)

In [10]:
with open("../ressources/automode_evaluated_concat_s14s15s16_n300_24-12-18.pickle", "rb") as file:
    original_df = pickle.load(file)

In [11]:
# from pipeline import CustomSFTTrainer

# sft_trainer = CustomSFTTrainer()
# trained_model, hf_sft_trainer, dataset_train = sft_trainer.train()


In [12]:
# from pipeline import CustomInference

# generated_text = CustomInference(model=trained_model,tokenizer=sft_trainer.tokenizer).inference()
# generated_text

In [13]:
# from pipeline.utils import DEFAULT_GENERATE_PROMPT

# DEFAULT_GENERATE_PROMPT("test", "", sft_trainer)

In [14]:
from pipeline import CustomDPOTrainer

dpo_trainer = CustomDPOTrainer(learning_rate=1000.99, num_train_epochs=5)
dpo_trained_model, hf_dpo_trainer, dataset_train = dpo_trainer.train()
print(dpo_trained_model is None)

import random
import numpy as np
random.seed(42)
np.random.seed(42)
from pipeline import CustomInference
generated_text = CustomInference(model=dpo_trained_model,tokenizer=dpo_trainer.tokenizer).inference()
generated_text

/home/jj/miniforge3/envs/automode_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


model or tokenizer or bnb or lora not set, loading default model


`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 6/6 [00:42<00:00,  7.06s/it]


dataset not set, using dummy dataset


Tokenizing train dataset: 100%|██████████| 10/10 [00:00<00:00, 673.17 examples/s]


ValueError: Adapter reference_adapt not found.

In [8]:
random.seed(42)
np.random.seed(42)

generated_text = CustomInference().inference()
generated_text

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model or tokenizer not set, loading default model


Loading checkpoint shards: 100%|██████████| 6/6 [00:29<00:00,  4.92s/it]


text not set, using dummy text


"<s>[INST] The environment features a rectangle with dimensions 5.57 x 2.19 x 1.75.There are the following lights in the arena: ((-0.25, 0.57)). 16 robots are evenly spaced around the central point, spanning a radius of 0.96 m. In the surroundings, there exists a circle at [-0.71, 0.76] with a radius of 0.25 meters, exhibiting a white color, and another at [0.78, -1.25] with a radius of 0.27 meters in black. The objective for the robots is to transfer items from the white initial location to the black circle. \nGenerate the behavior tree that achieves the objective of this mission.[/INST] Sure, here's a basic behavior tree that could achieve the objective of transferring items from the white circle to the black circle:\n\n1. **Start**: This is the initial state where the robot is idle.\n\n2. **Detect White Circle**: The robot uses its sensors to detect the white circle. If the white circle is detected, it proceeds to the next state. If not, it continues to scan the environment.\n\n3. *

In [ ]:
base_model = CustomInference().model

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model or tokenizer not set, loading default model


Loading checkpoint shards: 100%|██████████| 6/6 [00:22<00:00,  3.76s/it]


text not set, using dummy text


In [10]:
type(dpo_trained_model), type(base_model)

(transformers.models.mistral.modeling_mistral.MistralForCausalLM,
 peft.peft_model.PeftModelForCausalLM)

In [11]:
dpo_trained_model.save_pretrained("dpo_trained")
base_model.save_pretrained("base_model")


In [ ]:
hf_dpo_trainer.model.save_pretrained("hfmodel")

In [15]:
hf_dpo_trainer.save_model("sm")

In [12]:
dpo_trainer.model.save_pretrained("unmerged")

In [ ]:
from pipeline.utils import load_default_model

model, tokenizer, lora,bnb = load_default_model()

model.save_pretrained("base_model")
tokenizer.save_pretrained("base_model")